In [1]:
from gutenbergpy.gutenbergcache import GutenbergCache

In [2]:
import os
import json
import pandas as pd
import numpy as np

In [3]:
GutenbergCache.create()

Cache already exists


In [4]:
cache = GutenbergCache.get_cache()

In [ ]:
# The number of ENGLISH authors who have at least 5 works 
query = """
    SELECT COUNT(*) FROM (
        SELECT a.authorid
        FROM book_authors a
        LEFT JOIN books b
        ON b.id = a.bookid
        LEFT JOIN authors u
        ON u.id = a.authorid
        WHERE b.languageid = 1
        AND u.name NOT IN ('Anonyme', 'Anonymous, Anonymous', 'Anonymous', 'Various, Various', 'Various', 'Verschillende')
        GROUP BY a.authorid
        HAVING COUNT(a.bookid) >= 5
    ) AS author_counts;
"""

cursor = cache.native_query(query)
for row in cursor:
    print(row)

In [ ]:
query = """
        SELECT u.name
        FROM book_authors a
        LEFT JOIN books b
        ON b.id = a.bookid
        LEFT JOIN authors u
        ON u.id = a.authorid
        WHERE b.languageid = 1
        AND u.name NOT IN ('Anonyme', 'Anonymous, Anonymous', 'Anonymous', 'Various, Various', 'Various', 'Verschillende')
        GROUP BY a.authorid
        HAVING COUNT(a.bookid) = 5
"""
cursor = cache.native_query(query)
for row in cursor:
    print(row[0])

In [ ]:
import string

query = """
        SELECT u.name
        FROM book_authors a
        LEFT JOIN books b
        ON b.id = a.bookid
        LEFT JOIN authors u
        ON u.id = a.authorid
        WHERE b.languageid = 1
        AND u.name NOT IN ('Anonyme', 'Anonymous, Anonymous', 'Anonymous', 'Various, Various', 'Various', 'Verschillende')
        GROUP BY a.authorid
        HAVING COUNT(a.bookid) = 5
"""
cursor = cache.native_query(query)

# Check if a string contains only ASCII characters
def is_ascii(s):
    return all(c in string.ascii_letters or c in string.whitespace for c in s)

for row in cursor:
    author_name = row[0].replace(",", "")  # Remove the comma in the author name
    if is_ascii(author_name):
        print(author_name)


In [ ]:
import string

query = """
        SELECT u.name
        FROM book_authors a
        LEFT JOIN books b
        ON b.id = a.bookid
        LEFT JOIN authors u
        ON u.id = a.authorid
        WHERE b.languageid = 1
        AND u.name NOT IN ('Anonyme', 'Anonymous, Anonymous', 'Anonymous', 'Various, Various', 'Various', 'Verschillende')
        GROUP BY a.authorid
        HAVING COUNT(a.bookid) = 5
"""
cursor = cache.native_query(query)

def is_ascii(s):
    return all(c in string.ascii_letters or c in string.whitespace for c in s)

english_authors_count = 0
for row in cursor:
    author_name = row[0].replace(",", "")
    if is_ascii(author_name):
        english_authors_count += 1

print(english_authors_count)


In [ ]:
# query to get 5 random works of each qulified authorid 
queryy = """
    SELECT b.authorid, t.name
    FROM titles t
    LEFT JOIN book_authors b 
    ON b.bookid = t.bookid
    WHERE b.authorid IN (
        SELECT ba.authorid
        FROM book_authors ba
        LEFT JOIN books b
        ON b.id = ba.bookid
        WHERE b.languageid = 1
        GROUP BY ba.authorid
        HAVING COUNT(ba.bookid) >= 5
    )
    ORDER BY RANDOM()
    LIMIT 5;
"""
cursor = cache.native_query(queryy)
for row in cursor:
    print(row)

In [5]:
# get random qulified authorid

def get_n_random_authorids(n):
    query = """
        SELECT a.authorid
        FROM book_authors a
        LEFT JOIN books b
        ON b.id = a.bookid
        LEFT JOIN authors u
        ON u.id = a.authorid
        WHERE b.languageid = 1
        AND u.name NOT IN ('Anonyme', 'Anonymous, Anonymous', 'Anonymous', 'Various, Various', 'Various', 'Verschillende')
        GROUP BY a.authorid
        HAVING COUNT(a.bookid) >= 5
    """
    
    cursor = cache.native_query(query)
    authorids = [row[0] for row in cursor]
    return np.random.choice(authorids, n, replace=False)


In [6]:
# authorids = get_n_random_authorids(1)
# authorids

In [7]:
# get authornames

def get_authornames(authorid):
    query = """
        SELECT a.name
        FROM authors a

        LEFT JOIN book_authors b 
        ON b.authorid = a.id

        WHERE b.authorid = {}
    """.format(authorid)
    
    cursor = cache.native_query(query)
    for authornames in cursor:
        return authornames[0]


In [8]:
# for authorid in authorids:
#     authornames = get_authornames(authorid)
#     print(authornames)

In [9]:
# get 5 random works of each qulified authorid 

def get_5_random_works(authorid):
    query2 = """
        SELECT b.authorid, a.name, s.gutenbergbookid, t.name
        FROM titles t

        LEFT JOIN books s 
        ON s.id = t.bookid
        LEFT JOIN book_authors b 
        ON b.bookid = s.id
        LEFT JOIN authors a 
        ON b.authorid = a.id


        WHERE b.authorid = {}
        ORDER BY RANDOM()
        LIMIT 5;
    """.format(authorid)

    cursor = cache.native_query(query2)
    gutenbergbookids = []
    for row in cursor:
        gutenbergbookids.append(row[2])
    return gutenbergbookids

In [10]:
# for authorid in authorids:
#     gutenbergbookids = get_5_random_works(authorid)
#     print(gutenbergbookids)

In [11]:
# get titles

def get_titles(gutenbergbookid):
    query2 = """
        SELECT t.name
        FROM titles t

        LEFT JOIN books s 
        ON s.id = t.bookid

        WHERE s.gutenbergbookid = {}
    """.format(gutenbergbookid)

    cursor = cache.native_query(query2)
    titles = []
    for row in cursor:
        titles.append(row)
    return titles

In [12]:
# for gutenbergbookid in gutenbergbookids:
#     titles = get_titles(gutenbergbookid)
#     print(titles)

In [13]:
# randomly get 10 sample sentences with line number from each work, 50 lines each author 

import random
import gutenbergpy
import gutenbergpy.textget
import nltk # To split the book by sentences

def get_text(id):
    raw_book = gutenbergpy.textget.get_text_by_id(id)
    clean_book = gutenbergpy.textget.strip_headers(raw_book)
    return clean_book, raw_book

def trim_book(clean_book, trim_pct=0.1):
    lines = clean_book.strip().decode('utf-8').replace('\n', ' ')
    n = len(lines)
    start = n * trim_pct // 1
    end = n * (1 - trim_pct) // 1
    return lines[int(start):int(end)]

# def get_sample_lines(trimmed):
#     non_empty_lines = [line for line in trimmed if line.strip()]
#     n = len(non_empty_lines)
#     sample_lines = random.sample(range(n), k=min(n, 10))
#     sample = [(i+1, non_empty_lines[i]) for i in sample_lines]
#     return sample

def get_sample_sentences(trimmed):
    sentences = nltk.sent_tokenize(trimmed)
    non_empty_sentences = [s for s in sentences if s.strip()]
    n = len(non_empty_sentences)
    sample_sentences = random.sample(range(n), k=min(n, 10))
    sample = [(i+1, non_empty_sentences[i]) for i in sample_sentences]
    return sample

# def get_sample_sentences(trimmed, authorid, bookid):
#     sentences = nltk.sent_tokenize(trimmed)
#     non_empty_sentences = [s for s in sentences if s.strip()]
#     n = len(non_empty_sentences)
#     sample_sentences = random.sample(range(n), k=min(n, 10))
#     sample = [{'authorid': authorid, 'bookid': bookid, 'line_number': i+1, 'text': non_empty_sentences[i]} for i in sample_sentences]
#     return sample



# for authorid in authorids:
#     gutenbergbookids = get_5_random_works(authorid)
#     for id in gutenbergbookids:
#         clean, raw = get_text(id)
#         trimmed = trim_book(clean)
#         sample = get_sample_lines(trimmed)
#         print(f"Sample of 10 lines from work {id}:")
#         print(sample)

# for authorid in authorids:
#     gutenbergbookids = get_5_random_works(authorid)
#     for id in gutenbergbookids:
#         clean, raw = get_text(id)
#         trimmed = trim_book(clean)
#         sample = get_sample_sentences(trimmed)
#         print(f"Sample of 10 non-empty sentences from gutenberg book id {id}:")
#         print(sample)

In [14]:
# Combine and consolidate

def get_consolidated_results(n):
    authorids = get_n_random_authorids(n)

    results = []
    
    for authorid in authorids:
        authornames = get_authornames(authorid)
        gutenbergbookids = get_5_random_works(authorid)
        for gutenbergbookid in gutenbergbookids:
            titles = get_titles(gutenbergbookid)
            clean_book, raw_book = get_text(gutenbergbookid)
            trimmed = trim_book(clean_book, trim_pct=0.1)
            sample_sentences = get_sample_sentences(trimmed)

            for line_number, sentence in sample_sentences:
                results.append({
                    'authorid': authorid,
                    'authorname': authornames,
                    'gutenbergbookid': gutenbergbookid,
                    'title': titles,
                    'line_number': line_number,
                    'text': sentence
                })
    
    df = pd.DataFrame(results)
    
    return df

In [15]:
df = get_consolidated_results(1)
df

,authorid,authorname,gutenbergbookid,title,line_number,text
0,3414,"Malory, Thomas, Sir",46853,[(Le Morte Darthur\nSir Thomas Malory's Book o...,12855,"And all that were with king Mark, that were co..."
1,3414,"Malory, Thomas, Sir",46853,[(Le Morte Darthur\nSir Thomas Malory's Book o...,4829,So they departed all save Gouvernail.
2,3414,"Malory, Thomas, Sir",46853,[(Le Morte Darthur\nSir Thomas Malory's Book o...,1006,"Sir, said the king, the knight with the two sw..."
3,3414,"Malory, Thomas, Sir",46853,[(Le Morte Darthur\nSir Thomas Malory's Book o...,5254,_How La Beale Isoud sent letters unto Sir Tris...
4,3414,"Malory, Thomas, Sir",46853,[(Le Morte Darthur\nSir Thomas Malory's Book o...,7084,And for this matter was the king passing wroth...
5,3414,"Malory, Thomas, Sir",46853,[(Le Morte Darthur\nSir Thomas Malory's Book o...,12639,"So the carter drove on a great wallop, and Sir..."
6,3414,"Malory, Thomas, Sir",46853,[(Le Morte Darthur\nSir Thomas Malory's Book o...,1611,Unto this were all the knights sworn of the Ta...
7,3414,"Malory, Thomas, Sir",46853,[(Le Morte Darthur\nSir Thomas Malory's Book o...,12272,"And when she was relieved she sighed, and said..."
8,3414,"Malory, Thomas, Sir",46853,[(Le Morte Darthur\nSir Thomas Malory's Book o...,959,CHAP.
9,3414,"Malory, Thomas, Sir",46853,[(Le Morte Darthur\nSir Thomas Malory's Book o...,12259,"Fie on him, said the queen, yet for all his pr..."
